In [233]:
import torch
import torch.nn as nn

class VectorQuantization(nn.Module):
    def __init__(self, num_embeddings, embedding_dim):
        super(VectorQuantization, self).__init__()
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        self.embedding.weight.data.uniform_(-1, 1)  # Inicialización de los embeddings

    def forward(self, x):
        # Redimensionar la entrada para que sea 2D (batch_size, seq_len)
        x = x.view(-1, self.embedding_dim)
        
        # Encontrar el índice más cercano en el diccionario de embeddings
        dist = torch.cdist(x.unsqueeze(0), self.embedding.weight.unsqueeze(0))
        indices = torch.argmin(dist, dim=-1)
        quantized = self.embedding(indices)
        
        # Reshape de vuelta a la forma original
        quantized = quantized.view(x.size())

        return quantized, indices

# Crear una instancia de VectorQuantization con enteros
num_embeddings = 5  # Número de embeddings (ajustado a 100 para 100 enteros diferentes)
embedding_dim = 1    # Dimensión de cada embedding (1 para enteros)
vq = VectorQuantization(num_embeddings, embedding_dim)
# Datos de entrada (números enteros)
input_data = torch.tensor([[3.0, 7.0, 1.0, 9.0, 5.0]])
max_i = torch.max(input_data)
input_data = input_data/max_i

# Cuantizar los enteros
quantized_data, indices = vq(input_data)

# Reconstruir los enteros originales a partir de los índices
reconstructed_data = vq.embedding.weight[indices]

# Imprimir los resultados
print("Datos Originales:", input_data)
print("Datos Cuantizados:", quantized_data)
print("Índices de Cuantización:", indices)
print("Datos Reconstruidos:", reconstructed_data)

Datos Originales: tensor([[0.3333, 0.7778, 0.1111, 1.0000, 0.5556]])
Datos Cuantizados: tensor([[ 0.7610],
        [ 0.7610],
        [-0.2906],
        [ 0.7610],
        [ 0.7610]], grad_fn=<ViewBackward0>)
Índices de Cuantización: tensor([[2, 2, 3, 2, 2]])
Datos Reconstruidos: tensor([[[ 0.7610],
         [ 0.7610],
         [-0.2906],
         [ 0.7610],
         [ 0.7610]]], grad_fn=<IndexBackward0>)
